## Overview

- Single main process - postmaster / postgres
- Backends - process per client connection
- Background Processes - Parallel execution / Customer user code (extensions)
- Auxiliary Processes - Maintenance

#### Postmaster
- Main supervisor
- Daemon process - Listens on a port
- Manage client authentication and connection creation
- Spins up backends for every connection (even unsuccessful ones)
- Creates the memory context mechanism - memory recursive data structure to avoid leaks
- Creates the shared memory
- Spins up other backend processes (Background, Auxiliary)


- Since each forked process consumes CPU cores and RAM, the number of clients connecting to the server simultaneously are limited by the available cores and RAM. Any new client requests after the server resources are exhausted will be declined. That's actually ok because too much connections are bad architecture anyway.
- Connection Pooling - initiate several connections to the server during the connection pooler startup and serve them to the client as the requests come in.



#### Backends
- Forked from postmaster
- Responsible for a single client connection - process all queries
- Allowed to operate on a single DB
- Consume a dedicated virtual memory

#### Background Processes
- Pool of workers
- Execute parallel execution plans
- Use a different memory system to communicate

#### Auxiliary Processes
A group of processes that perform maintenance operations on the cluster:
- Autovacuum launcher - Launches autovacuum workers to perform vacuum and analyze operations 
- Background writer - In charge of writing from shared memory to disk when it's time to free space in the shared memory
- Checkpointer - Flushes dirty pages to disk to recycle the WAL buffer and ensure durability (not made for performance reasons)
- Logger - Logs stuff
- Startup process - Startup following a crash
- WAL archiver - Creates copies of WAL files on disk
- WAL receiver - Runs on a replica and receives WAL from the primary server
- WAL writer - Flushes dirty WAL

#### Check the processes

In [ ]:
# With PS
ps -HAf

In [ ]:
# With PSTREE
pstree -p

In [ ]:
# With SQL, more informative
export PGHOST=db
export PGUSER=postgres
export PGDATABASE=postgres

psql << EOM 
    SELECT 
        datname,
        pid,
        usename,
        client_addr,
        client_port,
        state,
        query,
        backend_type
    FROM pg_stat_activity;
EOM

In [ ]:
# Show max connections
psql -c "SHOW max_connections;"